# Projeto Ciencia de Dados: Carteira de Ações com Inteligência Artificial

- Projeto Inspiração: https://medium.com/swlh/teaching-a-machine-to-trade-stocks-like-warren-buffett-part-i-445849b208c6
- Resumo Fundamentos Empresas BR: https://fundamentus.com.br/index.php

### Criar Dicionários de Dataframes com cada empresa

In [52]:
import pandas as pd
import os

empresas = ["ABEV3", "AZUL4", "BTOW3", "B3SA3", "BBSE3", "BRML3", "BBDC4", "BRAP4", "BBAS3", "BRKM5", "BRFS3", "BPAC11", "CRFB3", "CCRO3", "CMIG4", "HGTX3", "CIEL3", "COGN3", "CPLE6", "CSAN3", "CPFE3", "CVCB3", "CYRE3", "ECOR3", "ELET6", "EMBR3", "ENBR3", "ENGI11", "ENEV3", "EGIE3", "EQTL3", "EZTC3", "FLRY3", "GGBR4", "GOAU4", "GOLL4", "NTCO3", "HAPV3", "HYPE3", "IGTA3", "GNDI3", "ITSA4", "ITUB4", "JBSS3", "JHSF3", "KLBN11", "RENT3", "LCAM3", "LAME4", "LREN3", "MGLU3", "MRFG3", "BEEF3", "MRVE3", "MULT3", "PCAR3", "PETR4", "BRDT3", "PRIO3", "QUAL3", "RADL3", "RAIL3", "SBSP3", "SANB11", "CSNA3", "SULA11", "SUZB3", "TAEE11", "VIVT3", "TIMS3", "TOTS3", "UGPA3", "USIM5", "VALE3", "VVAR3", "WEGE3", "YDUQ3"]
# fundamentos = {
#     "ABEV3": balanco_dre_abev3,
#     "MGLU3": balanco_dre_mglu3
# }
fundamentos = {}
arquivos = os.listdir("balancos") # Cria uma lista com todos os arquivos do diretório balanços
for arquivo in arquivos:
    nome = arquivo[-9:-4]
    if "11" in nome:
        nome = arquivo[-10:-4]
    if nome in empresas: # Validação para importar apenas empresas que estejam na lista empresas
        print(nome)
        balanco = pd.read_excel(f"balancos/{arquivo}", sheet_name=0)
        # Colocar o nome da empresa no cabeçalho
        balanco.iloc[0, 0] = nome
        # Tornando a primeira linha da tabela em cabeçalho
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]
        # Alterar o indice da tabela
        balanco = balanco.set_index(nome)
        dre = pd.read_excel(f"balancos/{arquivo}", sheet_name=1)
        # Colocar o nome da empresa no cabeçalho
        dre.iloc[0, 0] = nome
        # Tornando a primeira linha da tabela em cabeçalho
        dre.columns = dre.iloc[0]
        dre = dre[1:]
        # Alterar o indice da tabela
        dre = dre.set_index(nome)
        fundamentos[nome] = balanco._append(dre)
        # display(balanco)
        # display(dre)
        # break

ABEV3
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
AZUL4
WARNING *** file size (30309) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (30309) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
B3SA3
WARNING *** file size (64274) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64274) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
BBAS3
WARNING *** file size (68412) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (68412) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 

### Pegar Preços das Ações nas Datas Correspondentes

In [53]:
cotacoes_df = pd.read_excel("Cotacoes.xlsx")
cotacoes = {}
for empresa in cotacoes_df["Empresa"].unique():
    cotacoes[empresa] = cotacoes_df.loc[cotacoes_df["Empresa"] == empresa, :]
print(len(empresas))

77


### Remover empresas que tem cotações vazias da análise (mesmo após o tratamento que fizemos na hora de pegar as cotações)

In [54]:
for empresa in empresas:
    if cotacoes[empresa].isnull().values.any():
        cotacoes.pop(empresa)
        fundamentos.pop(empresa)
empresas = list(cotacoes.keys())
print(len(empresas))

65


### Juntar fundamentos com Preço da Ação

In [55]:
for empresa in empresas:
    tabela = fundamentos[empresa].T # Troca linhas pelas colunas utilizando T(transpose)
    tabela.index = pd.to_datetime(tabela.index, format="%d/%m/%Y") # Passa data(string) para o formato Python
    tabela_cotacao = cotacoes[empresa].set_index("Date")
    tabela_cotacao = tabela_cotacao[["Adj Close"]] # Deixando apenas uma coluna na tabela

    tabela = tabela.merge(tabela_cotacao, right_index=True, left_index=True) # Introduzindo apenas as linhas que tem nas duas tabelas
    tabela.index.name = empresa
    fundamentos[empresa] = tabela

### Tratar colunas
    
1. Vamos pegar apenas empresas que possuem as mesmas colunas
2. Ajeitar colunas com nome repetido
3. Analisar valores vazios nas colunas

#### 1. Remover da análise tabelas que possuem colunas diferentes

In [56]:
colunas = list(fundamentos["ABEV3"].columns)

for empresa in empresas:
    if set(colunas) != set(fundamentos[empresa].columns):
        fundamentos.pop(empresa)
print(len(fundamentos))

61


####  2. Ajeitando colunas com o mesmo nome

In [57]:
texto_colunas = ";".join(colunas)

colunas_modificadas = []
for coluna in colunas:
    if colunas.count(coluna) == 2:
        texto_colunas = texto_colunas.replace(";" + coluna + ";",";" + coluna + "_1;") # Adiciona _1 em colunas repetidas
        colunas_modificadas.append(coluna)
colunas = texto_colunas.split(";")
print(colunas)

['Ativo Total', 'Ativo Circulante', 'Caixa e Equivalentes de Caixa', 'Aplicações Financeiras', 'Contas a Receber_1', 'Estoques_1', 'Ativos Biológicos_1', 'Tributos a Recuperar', 'Despesas Antecipadas_1', 'Outros Ativos Circulantes', 'Ativo Realizável a Longo Prazo', 'Aplicações Financeiras Avaliadas a Valor Justo', 'Aplicações Financeiras Avaliadas ao Custo Amortizado', 'Contas a Receber_1', 'Estoques_1', 'Ativos Biológicos_1', 'Tributos Diferidos_1', 'Despesas Antecipadas_1', 'Créditos com Partes Relacionadas', 'Outros Ativos Não Circulantes', 'Investimentos', 'Imobilizado', 'Intangível', 'Diferido', 'Passivo Total', 'Passivo Circulante', 'Obrigações Sociais e Trabalhistas', 'Fornecedores', 'Obrigações Fiscais', 'Empréstimos e Financiamentos_1', 'Passivos com Partes Relacionadas_1', 'Dividendos e JCP a Pagar', 'Outros_1', 'Provisões_1', 'Passivos sobre Ativos Não-Correntes a Venda e Descontinuados_1', 'Passivo Não Circulante', 'Empréstimos e Financiamentos_1', 'Passivos com Partes Rel

In [58]:
for empresa in fundamentos:
    fundamentos[empresa].columns = colunas

#### 3. Analisar valores vazios nas colunas

In [70]:
valores_vazios = dict.fromkeys(colunas, 0) # Cria um dicionario com chaves=colunas e o valor inicial = 0
total_linhas = 0
for empresa in fundamentos:
    tabela = fundamentos[empresa]
    total_linhas += tabela.shape[0]
    for coluna in colunas:
        qntd_vazios = pd.isnull(tabela[coluna]).sum() # Soma a quantidade de valores vazios dentro da coluna
        valores_vazios[coluna] += qntd_vazios
print(valores_vazios)
print(total_linhas)

{'Ativo Total': 0, 'Ativo Circulante': 0, 'Caixa e Equivalentes de Caixa': 0, 'Aplicações Financeiras': 0, 'Contas a Receber_1': Contas a Receber_1    0
Contas a Receber_1    0
dtype: int64, 'Estoques_1': Estoques_1    0
Estoques_1    0
dtype: int64, 'Ativos Biológicos_1': Ativos Biológicos_1    0
Ativos Biológicos_1    0
dtype: int64, 'Tributos a Recuperar': 0, 'Despesas Antecipadas_1': Despesas Antecipadas_1    0
Despesas Antecipadas_1    0
dtype: int64, 'Outros Ativos Circulantes': 0, 'Ativo Realizável a Longo Prazo': 0, 'Aplicações Financeiras Avaliadas a Valor Justo': 0, 'Aplicações Financeiras Avaliadas ao Custo Amortizado': 0, 'Tributos Diferidos_1': Tributos Diferidos_1    0
Tributos Diferidos_1    0
dtype: int64, 'Créditos com Partes Relacionadas': 0, 'Outros Ativos Não Circulantes': 0, 'Investimentos': 0, 'Imobilizado': 0, 'Intangível': 0, 'Diferido': 0, 'Passivo Total': 0, 'Passivo Circulante': 0, 'Obrigações Sociais e Trabalhistas': 0, 'Fornecedores': 0, 'Obrigações Fiscais

In [74]:
remover_colunas = []
for coluna in valores_vazios:
    if (valores_vazios[coluna] > 50).all():
        remover_colunas.append(coluna)

for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].drop(remover_colunas, axis=1) # Excluindo colunas do eixo de colunas
    fundamentos[empresa] = fundamentos[empresa].ffill() # Substitui valores vazios com os valores da linha acima

In [75]:
fundamentos["ABEV3"].shape

(33, 72)

### Criando os rótulos: Comprar, Não Comprar ou Vender?

Não queremos saber quando vender, mas inclui essa categoria para conseguir identificar quando que o nosso modelo vai sugerir uma compra quando na verdade o melhor momento era vender. Isso significa que o modelo errou "mais" do que quando sugeriu comprar e simplesmente o certo era não comprar

Regra: 
1. Subiu mais do que o Ibovespa (ou caiu menos) -> Comprar (Valor = 2)
2. Subiu menos do que o Ibovespa até Ibovespa - 2% (ou caiu mais do que Ibovespa até Ibovespa -2%) -> Não Comprar (Valor = 1)
3. Subiu menos do que o Ibovespa - 2% (ou caiu mais do que Ibovespa -2%) -> Vender (Valor = 0)

### Hora de tornar tudo 1 dataframe só

### Análise Exploratória

#### 1. Quantidade de Respostas em cada Tipo de Decisão

#### 2. Correlação

#### Vamos remover Todas as Colunas "já explicadas" pelo Ativo Total

### Vamos partir para Feature Selection

Será que todas essas features são importantes mesmo para o nosso modelo? Muitas features nem sempre é bom, se pudermos reduzir sem perder eficiência do nosso modelo, melhor

Aqui temos 2 alternativas:

1. Seguir com todas as features e depois tentar melhorar o nosso modelo
2. Usar algum critério para selecionar as melhores features para prever e criar o modelo a partir apenas dessa seleção menor de features

Vou seguir com a opção 2, porque é mais rápida e, caso dê certo, facilita a nossa vida. Se der errado, a gente volta aqui e refaz o processo

### Aplicação do StandardScaler para melhorar nossos modelos de MachineLearning

### Separação dos dados em treino e teste

### Criação de um Dummy Classifier (Uma baseline para ver se os nossos modelos são melhores do que puro chute)

### Métricas de Avaliação

- Precisão vai ser nossa métrica principal
- Recall pode ser útil, mas precisão no caso de ações é mt mais importante.

Explicação: Foto dos Gatos e Cachorros na Wikipedia: https://en.wikipedia.org/wiki/Precision_and_recall

### Modelos que vamos testar
- AdaBoost
- Decision Tree
- Random Forest
- ExtraTree
- Gradient Boost
- K Nearest Neighbors (KNN)
- Logistic Regression
- Naive Bayes
- Support Vector Machine (SVM)
- Rede Neural

### Agora vamos ao tunning do modelo

- é bom sempre incluir no tuning os parâmetros "padrões" do modelo, se não poder ser que vc só encontre resultados piores

### Vamos seguir então com o Modelo BLA tunado para ser o nosso modelo

- Agora ao que interessa...será que vamos conseguir escolher boas ações e ter uma carteira rentável?
- Para isso, temos que pegar o último Trimestre (que não usamos para treinar o modelo nem testar) e ver como ele teria saído até o momento em que gravo esse vídeo...

### Repetindo os passos com a base fundamentos só que pegando apenas o último tri de cada empresa

### Agora fazemos a previsão de decisões